In [53]:
import pandas as pd
import wrds

In [89]:
conn = wrds.Connection(wrds_username='yjzhu')
conn.create_pgpass_file() 

Loading library list...
Done
Enter your WRDS username [admin]:yjzhu
Enter your password:········


In [90]:
import numpy as np
from scipy.stats.mstats import winsorize
from scipy import stats
import statsmodels.formula.api as sm
import patsy
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error

## Question1

In [1]:
#Sample Selection
funda = conn.raw_sql("""select exchg, gvkey, fyear,datadate, oiadp,ni, csho, at,act,che,lct,dlc,txp,dp,prcc_f,sich,dvt,lt,bkvlps, epsfx          
                         from compa.funda 
                         where consol='C' and indfmt='INDL' and datafmt='STD' and popsrc='D' and  fyear<2019 and fyear>2007
                         """) 
company = conn.raw_sql("""select gvkey, sic,conml,spcsrc
                          from compa.company
                          """) 
funda1=pd.merge(funda,company,on=['gvkey'])
funda1['sic1']=np.where(funda1['sich']>0,funda1['sich'],funda1['sic'])
funda1=funda1.drop(['sich','sic'],axis=1)
funda1['sic1']=funda1['sic1'].astype(str)

NameError: name 'conn' is not defined

In [92]:
funda1

,exchg,gvkey,fyear,datadate,oiadp,ni,csho,at,act,che,...,txp,dp,prcc_f,dvt,lt,bkvlps,epsfx,conml,spcsrc,sic1
0,11.0,001004,2008.0,2009-05-31,125.529,78.651,38.884,1377.511,851.312,112.505,...,6.482,40.551,14.7000,0.000,720.616,16.8937,1.92,AAR Corp,B,5080.0
1,11.0,001004,2009.0,2010-05-31,95.415,44.628,39.484,1501.042,863.429,79.370,...,3.263,38.930,19.7000,0.000,754.692,18.9167,1.16,AAR Corp,B,5080.0
2,11.0,001004,2010.0,2011-05-31,137.016,69.826,39.781,1703.727,913.985,57.433,...,0.000,59.296,26.3900,2.983,868.438,21.0112,1.81,AAR Corp,B,5080.0
3,11.0,001004,2011.0,2012-05-31,142.360,67.723,40.273,2195.653,1063.272,67.720,...,0.000,80.333,12.0500,12.081,1329.631,21.4697,1.65,AAR Corp,B,5080.0
4,11.0,001004,2012.0,2013-05-31,136.600,55.000,39.382,2136.900,1033.700,75.300,...,0.000,108.600,20.0600,11.900,1217.400,23.3254,1.38,AAR Corp,B,5080.0
5,11.0,001004,2013.0,2014-05-31,142.600,72.900,39.560,2199.500,1116.900,89.200,...,0.000,113.400,24.3000,11.800,1198.800,25.2654,1.83,AAR Corp,B,5080.0
6,11.0,001004,2014.0,2015-05-31,-8.600,10.200,35.423,1515.000,954.100,54.700,...,0.000,92.300,29.5400,11.900,669.900,23.8574,-1.40,AAR Corp,B,5080.0
7,11.0,001004,2015.0,2016-05-31,66.100,47.700,34.515,1442.100,873.100,31.200,...,1.100,70.800,24.4100,10.400,576.300,25.0847,1.16,AAR Corp,B,5080.0
8,11.0,001004,2016.0,2017-05-31,77.200,56.500,34.354,1504.100,888.500,10.300,...,12.300,71.000,34.9400,10.200,589.900,26.6112,1.45,AAR Corp,B,5080.0
9,11.0,001004,2017.0,2018-05-31,86.000,15.600,34.716,1524.700,942.700,41.600,...,0.000,40.500,44.6900,10.300,588.400,26.9703,2.11,AAR Corp,B,5080.0


In [93]:
#Generate Lag Variable
funda1_lag1 = funda1[['gvkey','fyear','at','act','che','lct','dlc','txp']].copy()
funda1_lag1['fyear'] = funda1_lag1['fyear']+1
funda1_lag1=funda1_lag1.rename(columns = {'at':'at_lag1','act':'act_lag1','che':'che_lag1','lct':'lct_lag1','dlc':'dlc_lag1',
                                       'txp':'txp_lag1'})
funda1=pd.merge(funda1, funda1_lag1, how = 'left', on = ['gvkey','fyear'])

In [94]:
#winsorize the variable
varlist  = ['at','at_lag1','act_lag1','che_lag1','lct_lag1','dlc_lag1','txp_lag1','act','che','lct','dlc','txp','oiadp','ni','csho','dp','dvt','lt','bkvlps']
for var in varlist:    
    funda1[var] = funda1[var].replace(np.Inf, np.nan)    
    funda1[var]=np.where(funda1[var].isnull(), np.nan, winsorize(funda1[var], limits=(0.01,0.01)))

In [95]:
#Construct other Variables
funda1['AverageTotalAsset'] = (funda1['at']+funda1['at_lag1'])/2
funda1['Earnings'] = funda1['oiadp']/funda1['AverageTotalAsset']
funda1['chg_ca']=funda1['act']-funda1['act_lag1']
funda1['chg_cash'] = funda1['che']-funda1['che_lag1']
funda1['chg_cl'] =funda1['lct']-funda1['lct_lag1']
funda1['chg_std'] = funda1['dlc']-funda1['dlc_lag1']
funda1['chg_tp'] = funda1['txp']-funda1['txp_lag1']
funda1['Accural_Comp'] = ((funda1['chg_ca']-funda1['chg_cash']) -(funda1['chg_cl']-funda1['chg_std']-funda1['chg_tp'])-funda1['dp'])/funda1['AverageTotalAsset']
funda1['be'] = funda1['at']-funda1['lt']
funda1['CashFlow']=funda1['Earnings']-funda1['Accural_Comp']

In [96]:
fundaclean1 = funda1[['gvkey','conml','spcsrc','fyear','datadate','epsfx','Accural_Comp','at','dvt','be','bkvlps','CashFlow','Earnings','sic1']].copy()

In [97]:
fundaclean1['dvt'] = fundaclean1['dvt'].replace(np.nan, 0)
fundaclean1['DD'] = np.where(fundaclean1['dvt']==0,0,1)
fundaclean1['Neg_E'] = np.where(fundaclean1['epsfx']<0,1,0)
fundaclean1['Neg_E1'] = np.where(fundaclean1['Earnings']<0,1,0)

In [98]:
# Create lead earnings
fundaclean1_lead1= fundaclean1[['gvkey','fyear','Earnings']].copy()
fundaclean1_lead1['fyear'] = fundaclean1_lead1['fyear']-1
fundaclean1_lead1=fundaclean1_lead1.rename(columns = {'Earnings':'Earnings_lead'})
fundaclean1=pd.merge(fundaclean1,fundaclean1_lead1, how = 'left', on = ['gvkey','fyear'])

In [99]:
# Create lead earnings
fundaclean1_lead1= fundaclean1[['gvkey','fyear','epsfx']].copy()
fundaclean1_lead1['fyear'] = fundaclean1_lead1['fyear']-1
fundaclean1_lead1=fundaclean1_lead1.rename(columns = {'epsfx':'Eps_lead'})
fundaclean1=pd.merge(fundaclean1,fundaclean1_lead1, how = 'left', on = ['gvkey','fyear'])

In [100]:
fundaclean1 = fundaclean1.dropna()

In [101]:
fundaclean1

,gvkey,conml,spcsrc,fyear,datadate,epsfx,Accural_Comp,at,dvt,be,bkvlps,CashFlow,Earnings,sic1,DD,Neg_E,Neg_E1,Earnings_lead,Eps_lead
1,001004,AAR Corp,B,2009.0,2010-05-31,1.16,-0.021397,1501.042,0.000,746.350,18.9167,0.087691,0.066294,5080.0,0,0,0,0.085508,1.81
2,001004,AAR Corp,B,2010.0,2011-05-31,1.81,-0.041990,1703.727,2.983,835.289,21.0112,0.127497,0.085508,5080.0,1,0,0,0.073017,1.65
3,001004,AAR Corp,B,2011.0,2012-05-31,1.65,0.005253,2195.653,12.081,866.022,21.4697,0.067764,0.073017,5080.0,1,0,0,0.063058,1.38
4,001004,AAR Corp,B,2012.0,2013-05-31,1.38,-0.045235,2136.900,11.900,919.500,23.3254,0.108292,0.063058,5080.0,1,0,0,0.065769,1.83
5,001004,AAR Corp,B,2013.0,2014-05-31,1.83,-0.034084,2199.500,11.800,1000.700,25.2654,0.099852,0.065769,5080.0,1,0,0,-0.004631,-1.40
6,001004,AAR Corp,B,2014.0,2015-05-31,-1.40,-0.124485,1515.000,11.900,845.100,23.8574,0.119855,-0.004631,5080.0,1,1,1,0.044706,1.16
7,001004,AAR Corp,B,2015.0,2016-05-31,1.16,-0.068445,1442.100,10.400,865.800,25.0847,0.113151,0.044706,5080.0,1,0,0,0.052406,1.45
8,001004,AAR Corp,B,2016.0,2017-05-31,1.45,-0.026882,1504.100,10.200,914.200,26.6112,0.079289,0.052406,5080.0,1,0,0,0.056788,2.11
22,001013,ADC Telecommunications Inc.,B,2009.0,2009-09-30,-4.78,-0.063775,1343.600,0.000,356.200,3.6874,0.074986,0.011211,3661.0,0,1,0,0.042511,0.78
32,001045,American Airlines Group Inc,B-,2017.0,2017-12-31,3.90,-0.038473,51396.000,198.000,3926.000,8.2564,0.131820,0.093348,4512.0,1,0,0,0.061602,3.03


In [102]:
fundaclean1.drop(fundaclean1[fundaclean1['fyear']==2016].index,inplace = True)

In [103]:
fundaclean1.drop(fundaclean1[fundaclean1['fyear']==2017].index,inplace = True)

In [104]:
fundaclean1=fundaclean1.rename(columns={'dvt':'D','bkvlps':'B','epsfx':'E','Accural_Comp':'AC','at':'A','sic1':'SIC'})

In [105]:
fundaclean1

,gvkey,conml,spcsrc,fyear,datadate,E,AC,A,D,be,B,CashFlow,Earnings,SIC,DD,Neg_E,Neg_E1,Earnings_lead,Eps_lead
1,001004,AAR Corp,B,2009.0,2010-05-31,1.16,-0.021397,1501.042,0.000,746.350,18.9167,0.087691,0.066294,5080.0,0,0,0,0.085508,1.81
2,001004,AAR Corp,B,2010.0,2011-05-31,1.81,-0.041990,1703.727,2.983,835.289,21.0112,0.127497,0.085508,5080.0,1,0,0,0.073017,1.65
3,001004,AAR Corp,B,2011.0,2012-05-31,1.65,0.005253,2195.653,12.081,866.022,21.4697,0.067764,0.073017,5080.0,1,0,0,0.063058,1.38
4,001004,AAR Corp,B,2012.0,2013-05-31,1.38,-0.045235,2136.900,11.900,919.500,23.3254,0.108292,0.063058,5080.0,1,0,0,0.065769,1.83
5,001004,AAR Corp,B,2013.0,2014-05-31,1.83,-0.034084,2199.500,11.800,1000.700,25.2654,0.099852,0.065769,5080.0,1,0,0,-0.004631,-1.40
6,001004,AAR Corp,B,2014.0,2015-05-31,-1.40,-0.124485,1515.000,11.900,845.100,23.8574,0.119855,-0.004631,5080.0,1,1,1,0.044706,1.16
7,001004,AAR Corp,B,2015.0,2016-05-31,1.16,-0.068445,1442.100,10.400,865.800,25.0847,0.113151,0.044706,5080.0,1,0,0,0.052406,1.45
22,001013,ADC Telecommunications Inc.,B,2009.0,2009-09-30,-4.78,-0.063775,1343.600,0.000,356.200,3.6874,0.074986,0.011211,3661.0,0,1,0,0.042511,0.78
34,001045,American Airlines Group Inc,B-,2009.0,2009-12-31,-4.99,-0.031178,25438.000,0.000,-3489.000,-10.4893,0.023718,-0.007460,4512.0,0,1,1,0.012192,-1.41
35,001045,American Airlines Group Inc,B-,2010.0,2010-12-31,-1.41,-0.046273,25088.000,0.000,-3945.000,-11.8309,0.058465,0.012192,4512.0,0,1,0,-0.007716,-5.91


In [106]:
# Random Walk Model
model1 = ols(formula ='Eps_lead~E', data = fundaclean1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Eps_lead   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.217
Method:                 Least Squares   F-statistic:                     6485.
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:44:36   Log-Likelihood:            -1.4995e+05
No. Observations:               23341   AIC:                         2.999e+05
Df Residuals:                   23339   BIC:                         2.999e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0203      0.977     -0.021      0.983      -1.935       1.895
E              2.8637      0.036     80.530      0.000       2.794       2.933
==============================================================================
Omnibus:                    96372.910   Durbin-Watson:                   1.365
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     206382370782.618
Skew:                         113.035   Prob(JB):                         0.00
Kurtosis:                   14568.651   Cond. No.                         27.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [107]:
pred1 = model1.predict()
MSE1 = mean_squared_error(fundaclean1['Eps_lead'],pred1)
MSE1

22264.186032005604

In [108]:
# HVZ model
model2 = ols(formula ='Earnings_lead~Earnings+A+D+DD+Neg_E1+AC', data = fundaclean1).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Earnings_lead   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     3423.
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:44:52   Log-Likelihood:                -58247.
No. Observations:               23341   AIC:                         1.165e+05
Df Residuals:                   23334   BIC:                         1.166e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0012      0.033     -0.037      0.971      -0.065       0.063
Earnings       0.5694      0.005    106.030      0.000       0.559       0.580
A           5.464e-07   2.41e-06      0.227      0.820   -4.17e-06    5.26e-06
D          -1.587e-05   7.29e-05     -0.218      0.828      -0.000       0.000
DD             0.0715      0.043      1.670      0.095      -0.012       0.155
Neg_E1        -0.3922      0.044     -8.923      0.000      -0.478      -0.306
AC             0.1403      0.005     26.751      0.000       0.130       0.151
==============================================================================
Omnibus:                    65581.329   Durbin-Watson:                   2.171
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       6493530741.935
Skew:                         -36.403   Prob(JB):                         0.00
Kurtosis:                    2585.937   Cond. No.                     4.75e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.75e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [109]:
pred2 = model2.predict()
MSE2 = mean_squared_error(fundaclean1['Earnings_lead'],pred2)
MSE2

8.611644808374612

In [110]:
#EP model
model3 = ols(formula ='Eps_lead~Neg_E+E+Neg_E*E',data = fundaclean1).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Eps_lead   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                 1.911e+04
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:44:56   Log-Likelihood:            -1.3834e+05
No. Observations:               23341   AIC:                         2.767e+05
Df Residuals:                   23337   BIC:                         2.767e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -17.9236      0.767    -23.371      0.000     -19.427     -16.420
Neg_E         17.5954      1.221     14.415      0.000      15.203      19.988
E              9.6641      0.040    239.344      0.000       9.585       9.743
Neg_E:E       -9.5413      0.048   -199.359      0.000      -9.635      -9.447
==============================================================================
Omnibus:                    32166.829   Durbin-Watson:                   1.170
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      11117673682.813
Skew:                          -5.973   Prob(JB):                         0.00
Kurtosis:                    3384.039   Cond. No.                         77.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [111]:
pred3 = model3.predict()
MSE3 = mean_squared_error(fundaclean1['Eps_lead'],pred3)
MSE3

8231.91548111613

In [112]:
#RI model
model4= ols(formula = 'Eps_lead~E+Neg_E+Neg_E:E+B+AC', data = fundaclean1).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Eps_lead   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                 1.149e+04
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:45:30   Log-Likelihood:            -1.3832e+05
No. Observations:               23341   AIC:                         2.767e+05
Df Residuals:                   23335   BIC:                         2.767e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -17.8635      0.766    -23.306      0.000     -19.366     -16.361
E              9.6700      0.040    239.579      0.000       9.591       9.749
Neg_E         17.5884      1.220     14.413      0.000      15.197      19.980
Neg_E:E       -9.5479      0.048   -199.577      0.000      -9.642      -9.454
B             -0.0026      0.000     -5.824      0.000      -0.004      -0.002
AC             0.0288      0.140      0.206      0.837      -0.246       0.303
==============================================================================
Omnibus:                    32123.204   Durbin-Watson:                   1.171
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      11055126495.207
Skew:                          -5.955   Prob(JB):                         0.00
Kurtosis:                    3374.515   Cond. No.                     2.97e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.97e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [113]:
pred4 = model4.predict()
MSE4 = mean_squared_error(fundaclean1['Eps_lead'],pred4)
MSE4

8219.950480350648

In [114]:
model5= ols(formula='Earnings_lead~CashFlow+AC', data = fundaclean1).fit()
model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Earnings_lead   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                 1.017e+04
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:45:45   Log-Likelihood:                -58301.
No. Observations:               23341   AIC:                         1.166e+05
Df Residuals:                   23338   BIC:                         1.166e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1042      0.019     -5.402      0.000      -0.142      -0.066
CashFlow       0.5767      0.005    108.068      0.000       0.566       0.587
AC             0.7145      0.005    135.211      0.000       0.704       0.725
==============================================================================
Omnibus:                    65454.826   Durbin-Watson:                   2.168
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       6440926777.624
Skew:                         -36.210   Prob(JB):                         0.00
Kurtosis:                    2575.457   Cond. No.                         5.20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [115]:
pred5 = model5.predict()
MSE5 = mean_squared_error(fundaclean1['Earnings_lead'],pred5)
MSE5

8.651573352520648